# MLFlow Basics

## O que você irá aprender aqui?

Este notebook utiliza um dataset didático para aprender os conceitos básicos de MLFlow:
* Criar um exeprimento de Machine Learning;
* Rastrear parâmetros e métricas das execuções do experimento;
* Dispor o modelo em uma REST API utilizando um dos modelos do exeprimento.

## Sobre o Dataset

Aqui utilizaremos um dataset bem conhecido da comunidade de Ciência de Dados, de modo que possamos focar diretamente nas
execução de exeprimentos.

O Dataset contém as seguintes colunas:

- carat: Peso do diamante.
- cut: Qualidade do corte do diamante (Fair, Good, Very Good, Premium, Ideal).
- color: Cor do diamante, variando de J (pior) a D (melhor).
- clarity: Grau de clareza do diamante, variando de (I1 (pior), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (melhor))
- depth: Profundidade total do diamante, expressa como a relação entre a profundidade e a largura do diamante = z / mean(x, y).
- table: Largura da parte superior do diamante em relação à sua parte mais larga (%).
- price: Preço do diamante em dólares americanos.
- x: Comprimento em milímetros.
- y: Largura em milímetros.
- z: Profundidade em milímetros.

O propósito é prever o preço do diamante dado suas caracteristicas.

## Importação de bibliotecas e carregamento do Dataset

In [1]:
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow
import mlflow.sklearn
import warnings

warnings.filterwarnings("ignore")

In [3]:
# Carregar o dataset
df = pd.read_csv('./data/diamonds.csv', index_col=0)

df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 53940 entries, 1 to 53940
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.5+ MB


In [5]:
df.describe()

,carat,depth,table,price,x,y,z
count,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
mean,0.797940,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


## Preparando os dados para a modelagem

Como o propósito é didático, não faremos nenhum estudo analítico avançado nos dados. Vamos aplicar as técnicas básicas de ML para variáveis categóricas e numéricas e depois aplicaremos alguns modelos com diversos parâmetros e comparar os resultados utilizando as métricas convencionais de modelos de regressão.

In [6]:
X = df.drop(columns=['price'])
y = df['price']

# Dividir o dataset em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento: OneHotEncoder para variáveis categóricas e StandardScaler para variáveis numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', OneHotEncoder(drop='first'), X.select_dtypes(include=['category']).columns)
    ]
)

In [35]:
# Criando objeto com as configurações do modelo, 
# model_config = {
#     'model': RandomForestRegressor,
#     'params': {
#         'random_state': 42,
#         'n_estimators': 50,
#         'max_depth': 5
#     }
# }

# model_config = {
#     'model': Ridge,
#     'params': {
#         'random_state': 42,
#         'alpha': 2
#     }
# }

model_config = {
    'model': Lasso,
    'params': {
        'random_state': 42,
        'alpha': 2
    }
}

In [36]:
pipeline = Pipeline(
    [
        ('preprocessor', preprocessor), 
        ('model', model_config['model'](**model_config['params']))
    ]
)

# Vamos dar um nome para as execuções do modelo para podermos localizar futuramente na interface do MLFlow
mlflow.set_experiment('diamonds-price-prediction')

with mlflow.start_run():
    model = pipeline.fit(X_train, y_train)
    
    # Fazer previsões em treino e teste
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    # Calcular métricas de treino
    mae_train = mean_absolute_error(y_train, y_pred_train)
    rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
    r2_train = r2_score(y_train, y_pred_train)

    # Calcular métricas de teste
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Logar os hiperparâmetros, o modelo e as métricas com MLflow
    mlflow.log_params(model_config['params'])
    
    mlflow.sklearn.log_model(model, 'model')
    
    mlflow.log_metric('mae_train', mae_train)
    mlflow.log_metric('mae_test', mae_test)

    mlflow.log_metric('rmse_train', rmse_train)
    mlflow.log_metric('rmse_test', rmse_test)
    
    mlflow.log_metric('r2_train', r2_train)
    mlflow.log_metric('r2_test', r2_test)
    
print(f'Modelo: {model_config["model"].__name__}')
print(f'Parâmetros: {model_config["params"]}')
print('Resultados do teste')
print(f'MAE: {mae_test}')
print(f'RMSE: {rmse_test}')
print(f'R²: {r2_test}')

Modelo: Lasso
Parâmetros: {'random_state': 42, 'alpha': 2}
Resultados do teste
MAE: 892.7886672697165
RMSE: 1497.7816386973604
R²: 0.858880604270661


## Visualizando os resultados na interface do MLFlow

Basta rodar no terminal o comando:

`mlflow ui --port 8100`

Lembrando que o comando deve ser executado na mesma pasta em que está o arquivo Jupyter Notebook ou .py que você executou o treinamento do modelo.

Agora você pode alterar o modelo e/ou os parâmetros do modelo na célula anterior para criar novas runs e monitorar depois na interface do MLFlow.

## Inserindo o modelo em uma REST API

Para servir um modelo basta coletar o ID de uma run de um exeprimento e executar no terminal:
 
`mlflow models serve -m "runs:/<run-id>/model" -p 1234`

Para testar, uitizamos a lib requests passando as informações de um novo diamante.

In [28]:

import requests

data = {"dataframe_records": [{"carat": 0.28, "cut": "Ideal", "color": "D", "clarity": "SI2", "depth": 69.5, "table": 88, "x": 7, "y": 5, "z": 6}]}

requests.post("http://localhost:1234/invocations", json=data).json()

{'predictions': [1287.5441294623583]}